In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import json
from datetime import datetime
import io
import os
import sys

%reload_ext autoreload
%autoreload 2

from IPython.display import Video as JupyterVideo
from IPython.display import HTML
from IPython.display import display
from os import system, name

# os.environ["CUDA_VISIBLE_DEVICES"] = '1'
os.environ['PYOPENGL_PLATFORM'] = 'osmesa'

import pose_pipeline
from pose_pipeline import *
from pose_pipeline.utils.jupyter import play, play_grid
pose_pipeline.set_environmental_variables()


In [ ]:
! echo $CUDA_VISIBLE_DEVICES

[Data organization](erd.png)

## Import test videos

In [ ]:
from pose_pipeline.utils.video_format import insert_local_video

test_videos_path = '/videos'
files = os.listdir(test_videos_path)

for f in files:
    insert_local_video(f, datetime.now(), os.path.join(test_videos_path, f), video_project='DEMO_TEST', skip_duplicates=True)

In [ ]:
filt = {'video_project': "DEMO_TEST"}

# List videos in database
VideoInfo.populate(filt)
Video & 'video_project="DEMO_TEST"'

In [ ]:
from pose_pipeline.pipeline import BottomUpBridging,BottomUpBridgingPerson

BottomUpBridging.populate(filt)

In [ ]:
BottomUpBridging()

In [ ]:
video_keys = (Video & 'video_project="DEMO_TEST"').fetch('KEY')

for v in video_keys:
    print(v)
    v["bottom_up_method_name"] = "Bridging_OpenPose"

    BottomUpMethod.insert1(v, skip_duplicates=True)

In [ ]:
BottomUpPeople.populate(filt)

In [ ]:
BottomUpPeople()

In [ ]:
BottomUpPeople & filt & 'bottom_up_method_name="Bridging_OpenPose"'

In [ ]:
BlurredVideo.populate()

In [ ]:
BlurredVideo()

In [ ]:
(TrackingBboxMethodLookup & 'tracking_method_name="MMPose_RTMPose_Coco_Wholebody"')

## Run tracking algorithm

After running this, use the GUI to annotate the video to indicate the subject of interest.

In [ ]:
# select tracking method

filt = 'video_project="DEMO_TEST" AND filename LIKE "m002%"'

keys = ((Video & filt)).fetch('KEY')

print(f'Missing tracking methods: {len(keys)}')
tracking_method = (TrackingBboxMethodLookup & 'tracking_method_name="MMDet_RTMDet_"').fetch1('tracking_method')
for key in keys:
    key['tracking_method'] = tracking_method
    TrackingBboxMethod.insert1(key, skip_duplicates=True)

In [ ]:
TrackingBboxMethod()

In [ ]:
TrackingBbox()

In [ ]:
# OpenPose.populate(filt)
TrackingBbox.populate(filt)

In [ ]:
TrackingBbox()

In [ ]:
BottomUpPeople & video_keys & 'bottom_up_method_name="Bridging_OpenPose"'

In [ ]:
# produce visualizations of these outputs used for annotation

BlurredVideo.populate(filt)
TrackingBboxVideo.populate(filt)

In [ ]:
from pose_pipeline.utils.annotation import annotate

filt = {'video_project': "DEMO_TEST"}

# Run annotation GUI
to_label = ((TrackingBboxVideo & filt) - PersonBboxValid).fetch('KEY')

print(f'{len(to_label)} found to annotate')

if len(to_label) > 0:

    key = to_label[0]

    annotate(key)

In [ ]:
PersonBboxValid()

In [ ]:
PersonBbox.populate(filt)

top_down_method = (TopDownMethodLookup & 'top_down_method_name="MMPose_RTMPose_Coco_Wholebody"').fetch1('top_down_method')

keys = ((PersonBbox & filt) - (TopDownMethod & {'top_down_method': top_down_method})).fetch('KEY')

print(f'Missing top down methods: {len(keys)}')
for key in keys:
    key['top_down_method'] = top_down_method
    TopDownMethod.insert1(key)


In [ ]:
# populate the selected analyses
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
TopDownPerson.populate(filt)

In [ ]:
# keypoints = (TopDownPerson & filt).fetch('keypoints')
# keypoints[0].shape

In [ ]:
TopDownPersonVideo.populate(filt)

In [ ]:
TopDownPersonVideo()

In [ ]:
# keys = (TopDownPersonVideo & filt).fetch('KEY')
# print(keys)
play(TopDownPersonVideo & {'filename':'m002_20231115_144552.23106539.mp4'} & {'top_down_method': 15})

In [ ]:
((PersonBbox & filt) - (TopDownMethod & {'top_down_method': top_down_method})).fetch('KEY')

In [ ]:
display(PersonBbox())
display(PersonBboxValid())
display(TopDownMethod())
display(TopDownPerson())

## Perform additional analysis on the selected subject

In [ ]:
# compute the final bounding box for subjects of interest
pose_pipeline.PersonBbox.populate(filt)

# select the top down approach to run
top_down_method = (pose_pipeline.TopDownMethodLookup & 'top_down_method_name="MMPose"').fetch1('top_down_method')
keys = ((pose_pipeline.PersonBbox & filt) - (pose_pipeline.TopDownMethod & {'smpl_method': smpl_method})).fetch('KEY')
print(f'Missing top down methods: {len(keys)}')
for key in keys:
    key['top_down_method'] = top_down_method
    pose_pipeline.TopDownMethod.insert1(key)

# select an SMPL method to run
smpl_method = (pose_pipeline.SMPLMethodLookup & 'smpl_method_name="PARE"').fetch1('smpl_method')
keys = ((pose_pipeline.PersonBbox & filt) - (pose_pipeline.SMPLMethod & {'smpl_method': smpl_method})).fetch('KEY')
print(f'Missing SMPL methods: {len(keys)}')
for key in keys:
    key['smpl_method'] = smpl_method
    pose_pipeline.SMPLMethod.insert1(key)

In [ ]:
# populate the selected analyses

TopDownPerson.populate(filt)
SMPLPerson.populate(filt)

## Produce and review visualizations

In [ ]:
TopDownPersonVideo.populate(filt)
SMPLPersonVideo.populate(filt)

In [ ]:
keys = (TopDownPersonVideo & filt).fetch('KEY')
play(TopDownPersonVideo & keys[0])